In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split as tts
from sklearn.linear_model import ElasticNet
from scipy.linalg import toeplitz
from sklearn.metrics import mean_squared_error as mse
import warnings
warnings.filterwarnings('ignore')

In [ ]:
device = torch.device("cpu")
dtype = torch.float64

In [ ]:
from google.colab import files
uploads = files.upload()

Saving cars.csv to cars.csv


In [ ]:
data = pd.read_csv('cars.csv')

## Question 1

In [ ]:
class SCADLinear(nn.Module):
    def __init__(self, input_size, alpha=3.7, lambda_val=0.1):
        super(SCADLinear, self).__init__()
        self.linear = nn.Linear(input_size, 1,bias=False, device=device,dtype=dtype)
        self.alpha = alpha
        self.lambda_val = lambda_val

    def forward(self, x):
        return self.linear(x)

    def get_coefficients(self):
      return self.linear.weight

    def scad_penalty(self, params):
        penalty = torch.tensor(0.)
        for param in params:
            penalty += self.scad(param)
        return penalty

    def scad(self, beta):
      beta = torch.abs(beta)
      # element wise comparisons will evaluate the conditions for each element
      condition1 = beta <= self.lambda_val
      condition2 = (beta >= self.lambda_val) & (beta <= self.alpha * self.lambda_val)
      condition3 = beta >= self.alpha * self.lambda_val
      penalty = torch.zeros_like(beta)
      # penalty terms according to Kim et al and Andrew Charles
      penalty += torch.where(condition1.detach(), self.lambda_val * beta, torch.zeros_like(beta))
      penalty += torch.where(condition2.detach(), ((self.alpha * self.lambda_val * (beta - self.lambda_val)) - 0.5 * (
          beta ** 2 - self.lambda_val ** 2)) / (self.alpha - 1), torch.zeros_like(beta))
      penalty += torch.where(condition3.detach(), ((0.5 * (self.alpha - 1) * self.lambda_val ** 2) +
                                                   self.lambda_val **2), torch.zeros_like(beta))
      return penalty.sum()

    def fit(self, x, y, num_epochs=1000, lr=0.001):
        optimizer = optim.Adam(self.parameters(), lr=lr)
        for epoch in range(num_epochs):
            optimizer.zero_grad()
            output = self.forward(x)
            mse_loss = nn.functional.mse_loss(output, y)
            reg_loss = self.scad_penalty(self.parameters())
            loss = mse_loss + reg_loss
            loss.backward()
            optimizer.step()
            if epoch % 100 == 0:
                print(f"Epoch {epoch}, Loss: {loss.item()}")

    def predict(self, x):
        with torch.no_grad():
            return self.forward(x)


In [ ]:
x = data.loc[:,'CYL':'WGT'].values
y = data['MPG'].values
X = torch.from_numpy(x)
y = torch.from_numpy(y)

x_train, x_test, y_train, y_test = tts(X, y, test_size=0.2, random_state=1693)

model = SCADLinear(input_size=x_train.shape[1], lambda_val=0.1, alpha=3)
model.fit(x_train, y_train)

y_pred = model.predict(x_test.unsqueeze(0))
mse = nn.functional.mse_loss(y_pred, y_test)
print('The mean mse is:', mse.item())
#print("Predicted values:", y_pred)

print(model.get_coefficients())

Epoch 0, Loss: 668248.0878371521
Epoch 100, Loss: 261212.84540602553
Epoch 200, Loss: 80978.27876054416
Epoch 300, Loss: 19981.010602177164
Epoch 400, Loss: 4780.678353658208
Epoch 500, Loss: 2045.5576022751945
Epoch 600, Loss: 1682.0732268097433
Epoch 700, Loss: 1634.5365815335845
Epoch 800, Loss: 1615.5775521371183
Epoch 900, Loss: 1597.2302824522887
The mean mse is: 1657.011997832451
Parameter containing:
tensor([[ 0.07,  0.62, -0.04]], dtype=torch.float64, requires_grad=True)


## Question 2

In [ ]:
class ElasticNet(nn.Module):
    def __init__(self, input_size, alpha=1.0, l1_ratio=0.5):
        super(ElasticNet, self).__init__()
        self.input_size = input_size
        self.alpha = alpha
        self.l1_ratio = l1_ratio

        # Define the linear regression layer
        self.linear = nn.Linear(input_size, 1,bias=False,device=device,dtype=dtype)

    def forward(self, x):
      return self.linear(x)

    def loss(self, y_pred, y_true):
        mse_loss = nn.MSELoss()(y_pred, y_true)
        l1_reg = torch.norm(self.linear.weight, p=1)
        l2_reg = torch.norm(self.linear.weight, p=2)

        objective = (1/2) * mse_loss + self.alpha * (
            self.l1_ratio * l1_reg + (1 - self.l1_ratio) * (1/2)*l2_reg**2)

        return objective

    def fit(self, X, y, num_epochs=100, learning_rate=0.01):
        optimizer = optim.SGD(self.parameters(), lr=learning_rate)

        for epoch in range(num_epochs):
            self.train()
            optimizer.zero_grad()
            y_pred = self(X)
            loss = self.loss(y_pred, y)
            loss.backward()
            optimizer.step()

            if (epoch + 1) % 100 == 0:
                print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item()}")

    def predict(self, X):
        self.eval()
        with torch.no_grad():
            y_pred = self(X)
            return y_pred
    def get_coefficients(self):
        return self.linear.weight


In [ ]:
class SqrtLasso(nn.Module):
    def __init__(self, input_size, alpha=0.1):
        super(SqrtLasso, self).__init__()
        self.input_size = input_size
        self.alpha = alpha

        # Define the linear regression layer
        self.linear = nn.Linear(input_size, 1,bias=False,device=device,dtype=dtype)

    def forward(self, x):
        return self.linear(x)

    def loss(self, y_pred, y_true):
        mse_loss = nn.MSELoss()(y_pred, y_true)
        l1_reg = torch.norm(self.linear.weight, p=1,dtype=torch.float64)
        # l2_reg = torch.norm(self.linear.weight, p=2,dtype=torch.float64)

        loss = torch.sqrt(mse_loss) + self.alpha * (l1_reg)

        return loss

    def fit(self, X, y, num_epochs=200, learning_rate=0.01):
        optimizer = optim.Adam(self.parameters(), lr=learning_rate)

        for epoch in range(num_epochs):
            self.train()
            optimizer.zero_grad()
            y_pred = self(X)
            loss = self.loss(y_pred, y)
            loss.backward()
            optimizer.step()

            if (epoch + 1) % 100 == 0:
                print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item()}")

    def predict(self, X):
        self.eval()
        with torch.no_grad():
            y_pred = self(X)
        return y_pred

    def get_coefficients(self):
        return self.linear.weight

In [ ]:
def make_correlated_features(num_samples,p,rho):
  vcor = []
  for i in range(p):
    vcor.append(rho**i)
  r = toeplitz(vcor)
  mu = np.repeat(0,p)
  x = np.random.multivariate_normal(mu, r, size=num_samples)
  return x


rho =0.9
p = 20
n = 500
vcor = []
for i in range(p):
  vcor.append(rho**i)

x = make_correlated_features(n,p,rho)

In [ ]:
# from class
beta =np.array([10,7,4,-1,2,3,0,0,0,0,2,-1,3,2])
beta = beta.reshape(-1,1)
betastar = np.concatenate([beta,np.repeat(0,p-len(beta)).reshape(-1,1)],axis=0)
y = x@betastar + 0.5*np.random.normal(size=(n,1))

In [ ]:
x = torch.tensor(x,device=device)
y = torch.tensor(y,device=device)

In [ ]:
model1 = ElasticNet(input_size=x.shape[1], alpha=0.01,l1_ratio=0.5)

In [ ]:
model1.fit(x,y)
enet_pred = model1.predict(x)
mse1 = nn.MSELoss()
mse1 = mse1(enet_pred, y)
print('The average mse for Elastic Net is:', mse1)

Epoch [100/100], Loss: 7.215653150274042
The average mse for Elastic Net is: tensor(13.39, dtype=torch.float64)


In [ ]:
model2 = SqrtLasso(input_size=x.shape[1], alpha=0.01)
model2.fit(x,y)
sqlasso_pred = model2.predict(x)
mse2 = nn.MSELoss()
mse2 = mse2(sqlasso_pred, y)
print('The average mse for Sqrt Lasso is:', mse2)

Epoch [100/200], Loss: 17.56797103847596
Epoch [200/200], Loss: 12.437392607776381
The average mse for Sqrt Lasso is: tensor(147.57, dtype=torch.float64)


In [ ]:
model3 = SCADLinear(input_size=x.shape[1], alpha=0.01)
model3.fit(x,y)
scad_pred = model3.predict(x)
mse3 = nn.MSELoss()
mse3 = mse3(scad_pred, y)
print('The average mse for SCAD is:', mse3)

Epoch 0, Loss: 755.2768453041725
Epoch 100, Loss: 688.2842688587493
Epoch 200, Loss: 627.6500114031916
Epoch 300, Loss: 573.2024467678683
Epoch 400, Loss: 524.2711128304784
Epoch 500, Loss: 480.48491041579535
Epoch 600, Loss: 441.3794541647013
Epoch 700, Loss: 406.5155299724952
Epoch 800, Loss: 375.4684013422615
Epoch 900, Loss: 347.82936440203855
The average mse for SCAD is: tensor(323.11, dtype=torch.float64)


In [ ]:
betastar = betastar.flatten()
betastar2 = torch.from_numpy(betastar)
betastar2

tensor([10,  7,  4, -1,  2,  3,  0,  0,  0,  0,  2, -1,  3,  2,  0,  0,  0,  0,
         0,  0])

In [ ]:
enet_pred = model1.get_coefficients()
enet_pred

Parameter containing:
tensor([[ 6.25,  5.67,  4.18,  2.68,  2.35,  1.94,  1.12,  0.56,  0.65,  0.38,
          0.95,  0.70,  0.99,  1.12,  0.34,  0.30,  0.40, -0.01, -0.14, -0.17]],
       dtype=torch.float64, requires_grad=True)

In [ ]:
sqlasso_pred = model2.get_coefficients()
sqlasso_pred

Parameter containing:
tensor([[ 1.93,  1.98,  1.74,  1.72,  2.00,  1.84,  1.47,  1.47,  1.47,  1.56,
          1.29,  1.47,  1.13,  1.19,  0.75,  0.53,  0.70,  0.28,  0.50, -0.17]],
       dtype=torch.float64, requires_grad=True)

In [ ]:
scad_pred = model3.get_coefficients()
scad_pred

Parameter containing:
tensor([[0.81, 0.73, 1.12, 0.89, 1.10, 0.69, 0.84, 1.06, 0.69, 0.92, 0.74, 0.96,
         0.78, 1.01, 0.54, 0.67, 0.57, 0.49, 0.80, 0.43]], dtype=torch.float64,
       requires_grad=True)

In [ ]:
torch.set_printoptions(precision=2)
torch.stack([enet_pred[0], sqlasso_pred[0], scad_pred[0], betastar2])

tensor([[ 6.25,  5.67,  4.18,  2.68,  2.35,  1.94,  1.12,  0.56,  0.65,  0.38,
          0.95,  0.70,  0.99,  1.12,  0.34,  0.30,  0.40, -0.01, -0.14, -0.17],
        [ 1.93,  1.98,  1.74,  1.72,  2.00,  1.84,  1.47,  1.47,  1.47,  1.56,
          1.29,  1.47,  1.13,  1.19,  0.75,  0.53,  0.70,  0.28,  0.50, -0.17],
        [ 0.81,  0.73,  1.12,  0.89,  1.10,  0.69,  0.84,  1.06,  0.69,  0.92,
          0.74,  0.96,  0.78,  1.01,  0.54,  0.67,  0.57,  0.49,  0.80,  0.43],
        [10.00,  7.00,  4.00, -1.00,  2.00,  3.00,  0.00,  0.00,  0.00,  0.00,
          2.00, -1.00,  3.00,  2.00,  0.00,  0.00,  0.00,  0.00,  0.00,  0.00]],
       dtype=torch.float64, grad_fn=<StackBackward0>)